# Dog_Cat_classification_v1_2026-02



## Startup cells

In [0]:
# Set environment variables for sagemaker_studio imports

import os
os.environ['DataZoneProjectId'] = 'b0gaf5ybaxqhwn'
os.environ['DataZoneDomainId'] = 'dzd-csxrgu7jawpejr'
os.environ['DataZoneEnvironmentId'] = '54gpuf4oihrns7'
os.environ['DataZoneDomainRegion'] = 'us-east-1'

# create both a function and variable for metadata access
_resource_metadata = None

def _get_resource_metadata():
    global _resource_metadata
    if _resource_metadata is None:
        _resource_metadata = {
            "AdditionalMetadata": {
                "DataZoneProjectId": "b0gaf5ybaxqhwn",
                "DataZoneDomainId": "dzd-csxrgu7jawpejr",
                "DataZoneEnvironmentId": "54gpuf4oihrns7",
                "DataZoneDomainRegion": "us-east-1",
            }
        }
    return _resource_metadata
metadata = _get_resource_metadata()

In [0]:
"""
Logging Configuration

Purpose:
--------
This sets up the logging framework for code executed in the user namespace.
"""

from typing import Optional


def _set_logging(log_dir: str, log_file: str, log_name: Optional[str] = None):
    import os
    import logging
    from logging.handlers import RotatingFileHandler

    level = logging.INFO
    max_bytes = 5 * 1024 * 1024
    backup_count = 5

    # fallback to /tmp dir on access, helpful for local dev setup
    try:
        os.makedirs(log_dir, exist_ok=True)
    except Exception:
        log_dir = "/tmp/kernels/"

    os.makedirs(log_dir, exist_ok=True)
    log_path = os.path.join(log_dir, log_file)

    logger = logging.getLogger() if not log_name else logging.getLogger(log_name)
    logger.handlers = []
    logger.setLevel(level)

    formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")

    # Rotating file handler
    fh = RotatingFileHandler(filename=log_path, maxBytes=max_bytes, backupCount=backup_count, encoding="utf-8")
    fh.setFormatter(formatter)
    logger.addHandler(fh)

    logger.info(f"Logging initialized for {log_name}.")


_set_logging("/var/log/computeEnvironments/kernel/", "kernel.log")
_set_logging("/var/log/studio/data-notebook-kernel-server/", "metrics.log", "metrics")

In [0]:
import logging
from sagemaker_studio import ClientConfig, sqlutils, sparkutils, dataframeutils

logger = logging.getLogger(__name__)
logger.info("Initializing sparkutils")
spark = sparkutils.init()
logger.info("Finished initializing sparkutils")

In [0]:
def _reset_os_path():
    """
    Reset the process's working directory to handle mount timing issues.
    
    This function resolves a race condition where the Python process starts
    before the filesystem mount is complete, causing the process to reference
    old mount paths and inodes. By explicitly changing to the mounted directory
    (/home/sagemaker-user), we ensure the process uses the correct, up-to-date
    mount point.
    
    The function logs stat information (device ID and inode) before and after
    the directory change to verify that the working directory is properly
    updated to reference the new mount.
    
    Note:
        This is executed at module import time to ensure the fix is applied
        as early as possible in the kernel initialization process.
    """
    try:
        import os
        import logging

        logger = logging.getLogger(__name__)
        logger.info("---------Before------")
        logger.info("CWD: %s", os.getcwd())
        logger.info("stat('.'): %s %s", os.stat('.').st_dev, os.stat('.').st_ino)
        logger.info("stat('/home/sagemaker-user'): %s %s", os.stat('/home/sagemaker-user').st_dev, os.stat('/home/sagemaker-user').st_ino)

        os.chdir("/home/sagemaker-user")

        logger.info("---------After------")
        logger.info("CWD: %s", os.getcwd())
        logger.info("stat('.'): %s %s", os.stat('.').st_dev, os.stat('.').st_ino)
        logger.info("stat('/home/sagemaker-user'): %s %s", os.stat('/home/sagemaker-user').st_dev, os.stat('/home/sagemaker-user').st_ino)
    except Exception as e:
        logger.exception(f"Failed to reset working directory: {e}")

_reset_os_path()

## Notebook

In [0]:
#downloading data from kagel

!pip install kagglehub


In [0]:
import tensorflow as tf

train_dir = "/home/sagemaker-user/.cache/kagglehub/datasets/tongpython/cat-and-dog/versions/1/training_set/training_set"
val_dir   = "/home/sagemaker-user/.cache/kagglehub/datasets/tongpython/cat-and-dog/versions/1/test_set/test_set"

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=(224, 224),
    batch_size=32
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir,
    image_size=(224, 224),
    batch_size=32
)

print(train_ds.class_names)


Found 8005 files belonging to 2 classes.


Found 2023 files belonging to 2 classes.


['cats', 'dogs']


In [0]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

base_model = MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)

base_model.trainable = False  # freeze pretrained layers

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation="relu")(x)
output = Dense(1, activation="sigmoid")(x)

model = Model(base_model.input, output)

model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.summary()


      0/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step

4202496/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

 Total params: 2,422,081 (9.24 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [0]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=3
)


Epoch 1/3


2026-02-07 11:42:08.421694: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 19267584 exceeds 10% of free system memory.
2026-02-07 11:42:08.466351: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 51380224 exceeds 10% of free system memory.
2026-02-07 11:42:08.467987: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 19267584 exceeds 10% of free system memory.
2026-02-07 11:42:08.525536: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 19267584 exceeds 10% of free system memory.
2026-02-07 11:42:08.598521: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 19267584 exceeds 10% of free system memory.


  1/251 ━━━━━━━━━━━━━━━━━━━━ 34:16 8s/step - accuracy: 0.4062 - loss: 0.8647

  2/251 ━━━━━━━━━━━━━━━━━━━━ 4:54 1s/step - accuracy: 0.4531 - loss: 0.8590 

  3/251 ━━━━━━━━━━━━━━━━━━━━ 4:46 1s/step - accuracy: 0.4861 - loss: 0.8395

...

 94/251 ━━━━━━━━━━━━━━━━━━━━ 3:00 1s/step - accuracy: 0.6227 - loss: 0.6605

 95/251 ━━━━━━━━━━━━━━━━━━━━ 2:59 1s/step - accuracy: 0.6231 - loss: 0.6601

 96/251 ━━━━━━━━━━━━━━━━━━━━ 2:57 1s/step - accuracy: 0.6234 - loss: 0.6597

 97/251 ━━━━━━━━━━━━━━━━━━━━ 2:56 1s/step - accuracy: 0.6238 - loss: 0.6593

 98/251 ━━━━━━━━━━━━━━━━━━━━ 2:55 1s/step - accuracy: 0.6241 - loss: 0.6589

In [0]:
model.save("cat_dog_model.keras")

In [0]:
import os

cwd = os.getcwd()
print("Current working directory:", cwd)

print(os.listdir(os.getcwd()))


Current working directory: /home/sagemaker-user
['.bashrc', '.ipython', '.user_packages', '.cache', '.keras', '.config', 'cat_dog_model.keras']


In [0]:
import boto3

s3 = boto3.client("s3")

bucket_name = "imageclassficationprojectdemo"
local_file = "/home/sagemaker-user/cat_dog_model.keras"
s3_key = "models/cat_dog_model.keras"

s3.upload_file(local_file, bucket_name, s3_key)

print("Model uploaded to S3 successfully!")


Model uploaded to S3 successfully!


In [0]:
import tensorflow as tf

model = tf.keras.models.load_model("cat_dog_model.keras")
print("Model loaded")


Model loaded


In [0]:
from PIL import Image
import numpy as np

img = Image.open("dog.2.jpg").convert("RGB").resize((224, 224))
img = np.array(img) / 255.0
img = np.expand_dims(img, axis=0)

pred = model.predict(img)[0][0]

print("Raw prediction value:", pred)
print("Prediction:", "Dog 🐶" if pred > 0.5 else "Cat 🐱")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


Raw prediction value: 0.5100756
Prediction: Dog 🐶


## Shutdown cells

In [0]:
"""
Stop spark session and associated Athena Spark session
"""

from IPython import get_ipython as _get_ipython
_get_ipython().user_ns["spark"].stop()